In [1]:
import numpy as np # 基础线性代数扩展包
import pandas as pd # 数据处理工具箱
df_bank = pd.read_csv("./train_90.csv") # 读取文件
df_bank.head() # 显示文件前5行

,geohash_id,date_id,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,...,F_28,F_29,F_30,F_31,F_32,F_33,F_34,F_35,active_index,consume_index
0,4885e281g,20230104,-0.711,-0.696,-0.794,-0.727,-0.747,-0.792,1.539,2.433,...,0.073,0.344,0.006,-0.446,-0.502,-0.456,-0.457,-0.830,69.306,63.78
1,4885e281g,20230105,-0.909,-0.903,-0.947,-0.844,-0.856,-0.908,-0.371,0.990,...,0.055,0.298,0.007,-0.523,-0.558,-0.533,0.113,-0.887,68.881,61.62
2,4885e281g,20230106,-0.920,-0.925,-0.923,-0.852,-0.853,-0.915,-0.334,0.792,...,0.067,0.324,0.006,-0.535,-0.564,-0.540,0.367,-1.021,69.738,61.03
3,4885e281g,20230107,-0.926,-0.931,-0.943,-0.837,-0.850,-0.907,-0.993,-0.006,...,0.076,0.276,0.010,-0.534,-0.554,-0.521,0.550,-0.211,68.721,62.02
4,4885e281g,20230108,-0.750,-0.764,-0.818,-0.749,-0.764,-0.816,1.116,1.447,...,0.079,0.328,0.008,-0.468,-0.500,-0.419,-0.236,0.644,69.960,64.62


In [2]:
X=df_bank.iloc[:,2:-2]
y=df_bank['consume_index']
X.head() #显示新的特征集

,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,F_10,...,F_26,F_27,F_28,F_29,F_30,F_31,F_32,F_33,F_34,F_35
0,-0.711,-0.696,-0.794,-0.727,-0.747,-0.792,1.539,2.433,-0.136,1.295,...,0.093,0.0,0.073,0.344,0.006,-0.446,-0.502,-0.456,-0.457,-0.830
1,-0.909,-0.903,-0.947,-0.844,-0.856,-0.908,-0.371,0.990,-0.935,0.479,...,0.098,0.0,0.055,0.298,0.007,-0.523,-0.558,-0.533,0.113,-0.887
2,-0.920,-0.925,-0.923,-0.852,-0.853,-0.915,-0.334,0.792,-0.532,0.334,...,0.086,0.0,0.067,0.324,0.006,-0.535,-0.564,-0.540,0.367,-1.021
3,-0.926,-0.931,-0.943,-0.837,-0.850,-0.907,-0.993,-0.006,-0.826,0.391,...,0.091,0.0,0.076,0.276,0.010,-0.534,-0.554,-0.521,0.550,-0.211
4,-0.750,-0.764,-0.818,-0.749,-0.764,-0.816,1.116,1.447,-0.547,0.939,...,0.099,0.0,0.079,0.328,0.008,-0.468,-0.500,-0.419,-0.236,0.644


In [3]:
from sklearn.model_selection import train_test_split # 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size=0.2, random_state=0)

In [4]:
from sklearn.tree import DecisionTreeClassifier # 导入决策树分类器
from sklearn.model_selection import GridSearchCV # 导入网格搜索工具
from sklearn.ensemble import AdaBoostClassifier # 导入AdaBoost模型
# from sklearn.metrics import (f1_score, confusion_matrix) # 导入评估标准
from sklearn.metrics import mean_squared_error, r2_score

dt = DecisionTreeClassifier() # 选择决策树分类器作为AdaBoost的基准算法
ada = AdaBoostClassifier(dt) # AdaBoost模型

In [6]:
# 使用网格搜索优化参数
ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
                  "base_estimator__splitter" :   ["best", "random"],
                  "base_estimator__random_state" :   [7,9,10,12,15],
                  "algorithm" : ["SAMME","SAMME.R"],
                  "n_estimators" :[1,2,5,10],
                  "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}
ada_gs = GridSearchCV(ada,param_grid = ada_param_grid, 
                        scoring="neg_mean_squared_error", n_jobs= 1, verbose = 1)
ada_gs.fit(X_train,y_train) # 拟合模型
ada_gs = ada_gs.best_estimator_ # 最佳模型
y_pred = ada_gs.predict(X_test) # 进行预测

Fitting 5 folds for each of 1120 candidates, totalling 5600 fits


c:\Users\JIN\Anaconda3\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5600 fits failed out of a total of 5600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2800 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\JIN\Anaconda3\envs\pytorch\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\JIN\Anaconda3\envs\pytorch\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 486, in fit
    return super().fit(X, y, sample_weight)
  File "c:\Users\JIN\Anaconda3\envs\pytorch\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 146, in fit
  

ValueError: Unknown label type: 'continuous'

In [ ]:
# 计算MSE和R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 输出模型评估结果和目标方程
print('MSE:', mse)
print('R-squared:', r2)

# print("Adaboost测试准确率: {:.2f}%".format(ada_gs.score(X_test, y_test)*100))
# print("Adaboost测试F1分数: {:.2f}%".format(f1_score(y_test, y_pred)*100))